# Analysis of Covariance (ANCOVA)

Analysis of covariance (ANCOVA) is a general linear model which blends ANOVA and regression. ANCOVA evaluates whether the means of a dependent variable (dv) are equal across levels of a categorical independent variable (between) often called a treatment, while statistically controlling for the effects of other continuous variables that are not of primary interest, known as covariates or nuisance variables (covar).

Pingouin uses statsmodels.regression.linear_model.OLS to compute the ANCOVA.

## Get info about subjects

In [47]:
import pandas as pd
from IPython.display import display
import os
from tqdm import tqdm
from pathlib import Path

import warnings

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", message=".*column_view.*")

force = True
anonymizer = True

root_dir = Path.cwd()


def anondir(path: Path, prefix=root_dir) -> Path:
    """Anonymize a directory path by replacing user-specific parts with <root>."""
    if not anonymizer:
        return path
    path_str = str(path).replace(str(prefix), "<living-park>")
    return Path(path_str)


display(f"Running in root dir: {anondir(root_dir)}")
stats_dir = Path(root_dir) / "stats_QCed" / "sampled"
print(f"Stats directory: {anondir(stats_dir)}")

output_dir = root_dir / "ancova"
output_dir.mkdir(parents=True, exist_ok=True)
print(f"Output directory: {anondir(output_dir)}")

'Running in root dir: <living-park>'

Stats directory: <living-park>/stats_QCed/sampled
Output directory: <living-park>/ancova


In [48]:
def get_cohort_stats():
    filename = root_dir / "cohort" / "longitudinal_cohort_qced.csv"
    df_clinical = pd.read_csv(filename)
    print(f"Load cohort stats: {anondir(filename)}")
    columns = [
        "PATNO",
        "first_visit",
        "second_visit",
        "PD_status",
        "SEX",
        "AGE_AT_VISIT",
        "UPDRS",
        "durationT2_T1_y",
    ]
    df_clinical["PD_status"] = df_clinical["dx_group"].replace(
        {"PD-non-MCI": "PD", "HC": "HC"}
    )
    df_clinical.rename(columns={"NP3TOT": "UPDRS"}, inplace=True)
    n_pd = df_clinical[df_clinical["PD_status"] == "PD"]["PATNO"].nunique()
    n_hc = df_clinical[df_clinical["PD_status"] == "HC"]["PATNO"].nunique()
    print(f"Number of PD-non-MCI subjects: {n_pd}")
    print(f"Number of HC subjects: {n_hc}")
    print(f"Total number of subjects: {df_clinical['PATNO'].nunique()}")
    return df_clinical[columns]


df_clinical = get_cohort_stats()

Load cohort stats: <living-park>/cohort/longitudinal_cohort_qced.csv
Number of PD-non-MCI subjects: 112
Number of HC subjects: 89
Total number of subjects: 201


In [49]:
def assert_number_of_repetitions(df, hemisphere=None, repetitions=26):
    """
    Assert that each subjects/region has exactly N repetitions.
    """
    groups = ["subjects", "region"] + (["hemisphere"] if hemisphere else [])
    grouped = df.groupby(groups).count() == repetitions
    assert (
        grouped.all().all()
    ), f"Not all subjects/regions have {repetitions} repetitions. {grouped[grouped == False].count()}"

## Code
### Create baseline dataframe

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import pingouin as pg


def _ensure_parent(path: Path) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)


def _cached(path: Path, force: bool = False) -> pd.DataFrame | None:
    if not force and path.exists():
        return pd.read_parquet(path)
    return None


def _require_cols(df: pd.DataFrame, cols: list[str]) -> None:
    miss = [c for c in cols if c not in df.columns]
    if miss:
        raise ValueError(f"Missing required columns: {miss}")


def get_visit_at_metric_df(
    metric: str,
    visit: int,
    hemisphere: bool,
    cohort_df: pd.DataFrame,
    stats_dir: Path,
    output_dir: Path,
    force: bool = False,
    verbose: bool = False,
) -> pd.DataFrame:
    """
    Long-form df for one visit:
    ['repetition','subject_visit','PD_status',('hemisphere',) 'region', metric,
     'AGE_AT_VISIT','SEX','durationT2_T1_y' (visit 2)]
    """
    if visit not in (1, 2):
        raise ValueError("visit must be 1 or 2")

    visit_name = "baseline" if visit == 1 else "longitudinal"
    cache = output_dir / f"ancova_longitudinal_visit-{visit_name}_df_{metric}.parquet"

    df = _cached(cache, force=force)
    if df is not None:
        if verbose:
            print(f"[visit {visit}] loaded cache: {cache}")
        return df

    # Load and prepare data
    src = stats_dir / f"{metric}.parquet"
    wide = pd.read_parquet(src)

    # Clean columns
    wide = wide.drop(columns=["PATNO_id"], errors="ignore")
    if hemisphere and "hemi" in wide.columns:
        wide = wide.rename(columns={"hemi": "hemisphere"})

    # Add repetition counter
    gcols = ["subject_visit"] + (["hemisphere"] if hemisphere else [])
    wide["repetition"] = wide.groupby(gcols).cumcount() + 1

    # Melt to long format
    id_vars = ["repetition", "subject_visit", "PD_status"] + (
        ["hemisphere"] if hemisphere else []
    )
    long = wide.melt(id_vars=id_vars, var_name="region", value_name=metric)
    long["region"] = long["region"].str.replace(f"_{metric}", "", regex=False)

    # Merge with clinical data
    visit_col = "first_visit" if visit == 1 else "second_visit"
    clinical_cols = [visit_col, "AGE_AT_VISIT", "SEX", "PD_status"] + (
        ["durationT2_T1_y"] if visit == 2 else []
    )

    base = long.merge(
        cohort_df[clinical_cols],
        left_on=["subject_visit", "PD_status"],
        right_on=[visit_col, "PD_status"],
        how="inner",
    ).drop(columns=[visit_col])

    # Convert to numeric
    numeric_cols = [metric, "AGE_AT_VISIT", "SEX"] + (
        ["durationT2_T1_y"] if visit == 2 else []
    )
    for col in numeric_cols:
        if col in base:
            base[col] = pd.to_numeric(base[col], errors="coerce")

    # Save cache
    _ensure_parent(cache)
    base.to_parquet(cache)
    if verbose:
        print(f"[visit {visit}] saved: {cache}  rows={len(base)}")
    return base


def compute_longitudinal_change(
    metric: str,
    hemisphere: bool,
    cohort_df: pd.DataFrame,
    stats_dir: Path,
    output_dir: Path,
    force: bool = False,
    verbose: bool = False,
) -> pd.DataFrame:
    """
    Join visit-1 and visit-2 and compute relative change:
        metric_change = (metric_next - metric_baseline) / metric_baseline
    """
    cache = output_dir / f"ancova_longitudinal_change_{metric}.parquet"
    out = _cached(cache, force=force)
    if out is not None:
        if verbose:
            print(f"[change] loaded cache: {cache}")
        return out

    # Get data for both visits
    v1 = get_visit_at_metric_df(
        metric, 1, hemisphere, cohort_df, stats_dir, output_dir, force, verbose
    )
    v2 = get_visit_at_metric_df(
        metric, 2, hemisphere, cohort_df, stats_dir, output_dir, force, verbose
    )

    if v1.empty or v2.empty:
        raise ValueError("missing visit data")

    # Extract PATNO from subject_visit
    def _patno(s: pd.Series) -> pd.Series:
        return s.astype(str).str.split("_").str[0]

    v1["PATNO"] = _patno(v1["subject_visit"])
    v2["PATNO"] = _patno(v2["subject_visit"])

    # Merge visits
    merge_keys = ["PATNO", "region", "repetition"] + (
        ["hemisphere"] if hemisphere else []
    )
    m = v1.merge(v2, on=merge_keys, suffixes=("_baseline", "_next"))

    if m.empty:
        raise ValueError("no matching records between visits")

    # Calculate relative change
    m[f"{metric}_change"] = (m[f"{metric}_next"] - m[f"{metric}_baseline"]) / m[
        f"{metric}_baseline"
    ]

    # Keep only visit-2 covariates
    m = m.drop(columns=[c for c in m.columns if c.endswith("_baseline")])
    m = m.rename(columns=lambda x: x.replace("_next", ""))

    # Save cache
    _ensure_parent(cache)
    m.to_parquet(cache)
    if verbose:
        print(f"[change] saved: {cache}  rows={len(m)}")
    return m


def compute_ancova(
    change_df: pd.DataFrame,
    metric: str,
    hemisphere: bool,
    output_dir: Path,
    force: bool = False,
    verbose: bool = False,
) -> pd.DataFrame:
    """
    ANCOVA per (repetition, region[, hemisphere]) with covariates AGE_AT_VISIT, SEX, durationT2_T1_y.
    Returns: ['repetition','region',('hemisphere',) 'F','p-val','np2','n']
    """
    cache = output_dir / f"ancova_longitudinal_{metric}.parquet"
    out = _cached(cache, force=force)
    if out is not None:
        if verbose:
            print(f"[ancova] loaded cache: {cache}")
        return out

    # Verify required columns
    req_cols = [
        f"{metric}_change",
        "PD_status",
        "AGE_AT_VISIT",
        "SEX",
        "repetition",
        "region",
        "durationT2_T1_y",
    ]
    if hemisphere:
        req_cols.append("hemisphere")
    _require_cols(change_df, req_cols)

    # Define grouping and covariate columns
    group_cols = ["repetition", "region"] + (["hemisphere"] if hemisphere else [])
    covars = ["AGE_AT_VISIT", "SEX", "durationT2_T1_y"]

    def _one_ancova(g: pd.DataFrame) -> pd.Series:
        """Run ANCOVA for one group."""
        # Prepare data
        cols_needed = [f"{metric}_change", "PD_status"] + covars
        data = g[cols_needed].copy()

        # Convert to appropriate types and drop NaN
        data[f"{metric}_change"] = pd.to_numeric(
            data[f"{metric}_change"], errors="coerce"
        )
        for c in covars:
            data[c] = pd.to_numeric(data[c], errors="coerce")
        data["PD_status"] = data["PD_status"].astype(str)
        data = data.dropna()

        # Check minimum requirements
        n = len(data)
        if (
            n < 4
            or data["PD_status"].nunique() < 2
            or any(data[c].nunique() < 2 for c in covars)
        ):
            return pd.Series({"F": np.nan, "p-val": np.nan, "np2": np.nan, "n": n})

        # Run ANCOVA
        try:
            anc = pg.ancova(
                data=data, dv=f"{metric}_change", between="PD_status", covar=covars
            )
            row = anc.loc[anc["Source"] == "PD_status"].iloc[0]
            return pd.Series(
                {
                    "F": float(row["F"]),
                    "p-val": float(row["p-val"]),
                    "np2": float(row["np2"]),
                    "n": n,
                }
            )
        except Exception:
            return pd.Series({"F": np.nan, "p-val": np.nan, "np2": np.nan, "n": n})

    # Apply ANCOVA to each group
    res = (
        change_df.groupby(group_cols, sort=False, dropna=False)
        .apply(_one_ancova, include_groups=False)
        .reset_index()
    )

    # Save cache
    _ensure_parent(cache)
    res.to_parquet(cache)
    if verbose:
        print(f"[ancova] saved: {cache}  tests={len(res)}")
    return res


def compute_significance(
    ancova_df: pd.DataFrame,
    hemisphere: bool = False,
    alpha: float = 0.05,
) -> pd.DataFrame:
    """
    Aggregate ANCOVA results per region (and hemisphere if present).
    """
    if ancova_df.empty:
        return pd.DataFrame()

    df = ancova_df.copy()

    pcol = "p-val"
    # Determine significance
    df["significant"] = df[pcol] < alpha

    # Define grouping columns
    gcols = ["region"]
    if hemisphere and "hemisphere" in df.columns:
        gcols.append("hemisphere")

    # Aggregate statistics
    agg = (
        df.groupby(gcols, dropna=False)
        .agg(
            n_tests=("F", "count"),
            n_significant=("significant", "sum"),
            proportion_significant=("significant", "mean"),
            mean_F=("F", "mean"),
            std_F=("F", "std"),
            min_F=("F", "min"),
            max_F=("F", "max"),
            mean_p=(pcol, "mean"),
            std_p=(pcol, "std"),
            min_p=(pcol, "min"),
            max_p=(pcol, "max"),
            mean_n=("n", "mean"),
            min_n=("n", "min"),
            max_n=("n", "max"),
        )
        .reset_index()
    )

    return agg.sort_values("proportion_significant", ascending=False)


def report_significance_df(
    metric: str,
    cohort_df: pd.DataFrame,
    stats_dir: Path,
    output_dir: Path,
    hemisphere: bool = True,
    alpha: float = 0.05,
    force: bool = False,
    verbose: bool = False,
) -> pd.DataFrame:
    """
    Full longitudinal ANCOVA pipeline for a single metric.
    Returns the aggregated significance DataFrame.
    """
    # Compute longitudinal change
    change_df = compute_longitudinal_change(
        metric=metric,
        hemisphere=hemisphere,
        cohort_df=cohort_df,
        stats_dir=stats_dir,
        output_dir=output_dir,
        force=force,
        verbose=verbose,
    )

    # Run ANCOVA
    ancova_df = compute_ancova(
        change_df=change_df,
        metric=metric,
        hemisphere=hemisphere,
        output_dir=output_dir,
        force=force,
        verbose=verbose,
    )

    # Compute significance summary
    sig = compute_significance(
        ancova_df=ancova_df,
        hemisphere=hemisphere,
        alpha=alpha,
    )

    out_csv = output_dir / f"ancova_longitudinal_{metric}.csv"
    _ensure_parent(out_csv)
    sig.to_csv(out_csv, index=False)

    return sig

### Compute partial correlation

### Get significant partial correlation among repetitions

## Cortical volume

In [51]:
significance_volume_df = report_significance_df(
    metric="volume",
    cohort_df=df_clinical,
    hemisphere=True,
    alpha=0.05,
    force=force,
    stats_dir=stats_dir,
    output_dir=output_dir,
)

In [52]:
significance_volume_df

,region,hemisphere,n_tests,n_significant,proportion_significant,mean_F,std_F,min_F,max_F,mean_p,std_p,min_p,max_p,mean_n,min_n,max_n
19,fusiform,rh,26,15,0.576923,3.850799,1.884225,0.294108,8.279442,0.107097,0.150358,0.004476,0.588246,192.0,192.0,192.0
20,inferiorparietal,lh,26,9,0.346154,3.677734,1.561448,0.755960,7.312698,0.084345,0.079473,0.007479,0.385710,192.0,192.0,192.0
18,fusiform,lh,26,7,0.269231,2.626725,1.602524,0.036479,5.052165,0.198916,0.225462,0.025763,0.848736,192.0,192.0,192.0
36,middletemporal,lh,26,5,0.192308,2.634507,1.730345,0.024982,6.855125,0.210576,0.238724,0.009563,0.874582,192.0,192.0,192.0
47,parstriangularis,rh,26,5,0.192308,2.304540,1.457155,0.251501,5.504211,0.200349,0.165546,0.020018,0.616610,192.0,192.0,192.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,temporalpole,rh,26,0,0.000000,0.329794,0.427909,0.000188,1.842431,0.668852,0.234748,0.176302,0.989085,192.0,192.0,192.0
76,transversetemporal,lh,26,0,0.000000,0.630963,0.703386,0.002017,2.775496,0.545951,0.262315,0.097391,0.964226,192.0,192.0,192.0
77,transversetemporal,rh,26,0,0.000000,0.730266,0.789955,0.032853,3.516676,0.484264,0.210634,0.062313,0.856364,192.0,192.0,192.0
78,visit,lh,0,0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0


## Cortical thickness

In [53]:
significance_thickness_df = report_significance_df(
    metric="thickness",
    cohort_df=df_clinical,
    hemisphere=True,
    alpha=0.05,
    force=force,
    stats_dir=stats_dir,
    output_dir=output_dir,
)

In [54]:
significance_thickness_df

,region,hemisphere,n_tests,n_significant,proportion_significant,mean_F,std_F,min_F,max_F,mean_p,std_p,min_p,max_p,mean_n,min_n,max_n
44,parsopercularis,lh,26,13,0.500000,4.136691,1.736255,1.891066,8.901930,0.064420,0.048718,0.003229,0.170727,192.0,192.0,192.0
28,isthmuscingulate,lh,26,2,0.076923,1.697048,1.597082,0.040378,6.530970,0.326088,0.254470,0.011397,0.840961,192.0,192.0,192.0
62,rostralanteriorcingulate,lh,26,2,0.076923,1.935651,1.259463,0.034582,4.975189,0.234918,0.180499,0.026902,0.852676,192.0,192.0,192.0
27,insula,rh,26,2,0.076923,1.825828,1.911970,0.002276,8.659470,0.321193,0.265956,0.003666,0.962003,192.0,192.0,192.0
43,parahippocampal,rh,26,1,0.038462,1.019186,1.244798,0.001600,5.332861,0.473455,0.287522,0.022020,0.968138,192.0,192.0,192.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,temporalpole,rh,26,0,0.000000,0.393825,0.666786,0.000815,3.307991,0.649512,0.229784,0.070543,0.977257,192.0,192.0,192.0
78,transversetemporal,lh,26,0,0.000000,0.276420,0.328620,0.000658,1.121221,0.679201,0.205928,0.291021,0.979566,192.0,192.0,192.0
79,transversetemporal,rh,26,0,0.000000,0.217724,0.382915,0.000225,1.613191,0.757576,0.224440,0.205622,0.988039,192.0,192.0,192.0
80,visit,lh,0,0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0


## Cortical surface area

In [55]:
significance_area_df = report_significance_df(
    metric="area",
    cohort_df=df_clinical,
    hemisphere=True,
    alpha=0.05,
    force=force,
    stats_dir=stats_dir,
    output_dir=output_dir,
)

In [56]:
significance_area_df

,region,hemisphere,n_tests,n_significant,proportion_significant,mean_F,std_F,min_F,max_F,mean_p,std_p,min_p,max_p,mean_n,min_n,max_n
31,lateraloccipital,rh,26,24,0.923077,8.197534,2.728315,2.352692,13.571855,0.013836,0.027233,0.000301,0.126757,192.0,192.0,192.0
22,inferiorparietal,lh,26,22,0.846154,5.776624,1.992849,1.924280,10.507027,0.030570,0.037139,0.001408,0.167037,192.0,192.0,192.0
10,cuneus,lh,26,16,0.615385,4.534120,2.126493,0.912078,9.606303,0.067766,0.085216,0.002239,0.340797,192.0,192.0,192.0
21,fusiform,rh,26,15,0.576923,4.433915,2.412174,0.094086,8.301685,0.108693,0.182561,0.004424,0.759387,192.0,192.0,192.0
45,parsopercularis,rh,26,13,0.500000,3.825745,2.214750,0.793403,8.407924,0.101887,0.099186,0.004183,0.374218,192.0,192.0,192.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,superiorfrontal,rh,26,0,0.000000,0.453229,0.497219,0.004778,1.853846,0.599999,0.239935,0.174975,0.944963,192.0,192.0,192.0
77,temporalpole,rh,26,0,0.000000,0.327448,0.448101,0.000243,1.996525,0.665589,0.224019,0.159323,0.987570,192.0,192.0,192.0
75,supramarginal,rh,26,0,0.000000,0.386906,0.749020,0.000003,3.099585,0.687120,0.246624,0.079947,0.998715,192.0,192.0,192.0
80,visit,lh,0,0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0


## Subcortical volume

In [57]:
significance_subcortical_volume_df = report_significance_df(
    metric="subcortical_volume",
    cohort_df=df_clinical,
    hemisphere=False,
    alpha=0.05,
    force=force,
    stats_dir=stats_dir,
    output_dir=output_dir,
)

In [58]:
significance_subcortical_volume_df

,region,n_tests,n_significant,proportion_significant,mean_F,std_F,min_F,max_F,mean_p,std_p,min_p,max_p,mean_n,min_n,max_n
5,BrainSegVol-to-eTIV,26,22,0.846154,4.366187,0.619350,3.093400,5.927165,0.040426,0.014008,0.015848,0.080246,192.0,192.0,192.0
3,Brain-Stem,26,13,0.500000,4.021438,1.328247,1.578715,6.454990,0.064517,0.056640,0.011877,0.210513,192.0,192.0,192.0
11,CC_Posterior,26,5,0.192308,2.140772,1.911179,0.005884,6.081714,0.321307,0.312564,0.014560,0.938938,192.0,192.0,192.0
1,4th-Ventricle,26,5,0.192308,2.558490,1.316301,1.061978,6.472568,0.144140,0.079657,0.011764,0.304096,192.0,192.0,192.0
48,Right-choroid-plexus,26,5,0.192308,2.417335,2.651398,0.051769,9.917249,0.285254,0.245934,0.001906,0.820261,192.0,192.0,192.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,rhCerebralWhiteMatterVol,26,0,0.000000,0.392752,0.341681,0.003934,1.169188,0.595770,0.200344,0.280961,0.950057,192.0,192.0,192.0
64,rhCortexVol,26,0,0.000000,0.576767,0.248451,0.208287,1.193184,0.464536,0.097667,0.276093,0.648643,192.0,192.0,192.0
65,rhSurfaceHoles,26,0,0.000000,0.353654,0.367344,0.004471,1.544806,0.617788,0.190151,0.215460,0.946762,192.0,192.0,192.0
66,subject,0,0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
